## word2vec으로 e,s,g 단어집 만들기
2022/12/29

- cbow, skip gram 둘다 진행
- e : 환경, 친환경 
- s : 사회, 근로
- g : 경영, 지배 

In [1]:
import gensim

In [2]:
import pandas as pd
from tqdm import tqdm
from gensim.models.word2vec import Word2Vec

- news 데이터로 단어집 만들기

In [3]:
# news data 불러오기
news=pd.read_csv('./news_tb.csv')
len(news)

21761

In [4]:
news.head()

,Unnamed: 0,ID,CmpID,Year,Title,Content
0,0,1,1129,2021,"포스코ICT, 카카오워크 통해 RPA 서비스 시작",포스코 의 업무자동화 소프트웨어 로봇인 솔루션과 카카오워크간 연동 서비스가 시작된다...
1,1,2,218,2021,"그리티, 특별관계자 지분변동",그리티는 문영우 및 특별관계자의 지분율이 에서 로 변동했다고 일 공시했다 한편 그리...
2,2,3,1129,2021,"포스코ICT, 2021 기술 컨퍼런스 개최",포스코대표 정덕균가 지난 일 올해 추진된 기술개발 성과를 공유하고 최신 기술동향과 ...
3,3,4,2,2021,"[코스닥 대표 인터뷰]임형섭 석경에이티 대표 ""25년 나노 소재 노하우로 5G 시장...",파이낸셜뉴스우리나라에서 년 처음 시작해 년 간 나노 소재를 하고 있습니다 등의 가치...
4,4,5,218,2021,"그리티, 특별관계자 지분변동",그리티는 문영우 및 특별관계자의 지분율이 에서 로 변동했다고 일 공시했다 한편 그리...


In [5]:
print(news.isnull().values.any()) #결측값 없음, 숫자, 영어 제거된 상태

False


In [6]:
#로컬에 저장한 mecab 파일 경로 지정해주기
# 형태소분석기 okt 활용 토큰화 작업
 
from konlpy.tag import Okt, Mecab

okt = Okt()

tokenized_data = []
for sentence in tqdm(news['Content']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    #stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    #tokenized_data.append(stopwords_removed_sentence)
    tokenized_data.append(tokenized_sentence)

100%|██████████| 21761/21761 [22:06<00:00, 16.40it/s]  


In [7]:
# 토큰화된 뉴스 데이터 학습시키기
#https://hoonzi-text.tistory.com/2 파라미터 정리 
# size : 벡터 차원수 ,
# window : 훈련시 앞 뒤로 고려하는 단어의 개수
# min_count :최소빈도수 , 해당 빈도수보다 작게 등장한 단어 제외
# worker : 병렬처리할 스레드 수 
#sg : 분석방법론 , 1 = skip-gram, 0=cbow
#iter 학습횟수

from gensim.models import Word2Vec

#cbow
model=Word2Vec(tokenized_data, vector_size=100, window=3, min_count=5,workers=4, sg=0,epochs=100)

In [8]:
model.wv.vectors.shape

(27429, 100)

In [9]:
e_친환경=model.wv.most_similar("친환경",topn=50)
e_환경=model.wv.most_similar("환경",topn=50)
s_사회=model.wv.most_similar("사회",topn=50)
s_근로=model.wv.most_similar("근로",topn=50)
g_경영=model.wv.most_similar("경영",topn=50)
g_지배=model.wv.most_similar("지배",topn=50)


In [10]:

def make_dict(df):
    temp = []
    for word, percent in df:
        temp.append(word)
    df = temp
    return df

e_친환경df=make_dict(e_친환경)
e_환경df=make_dict(e_환경)
s_근로df=make_dict(s_근로)
s_사회df=make_dict(s_사회)
g_지배df=make_dict(g_지배)
g_경영df=make_dict(g_경영)

In [11]:
esg_df=pd.DataFrame(zip(e_친환경df, e_환경df,s_근로df,s_사회df,g_지배df,g_경영df))
esg_df.columns=['e_친환경','e_환경','s_근로','s_사회','g_지배','g_경영']
esg_df.to_csv('./esg_dict_ver1.csv')

In [12]:
#모델 저장
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('news_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("news_w2v") # 모델 로드

In [13]:
# 토큰화된 뉴스 데이터 학습시키기
#https://hoonzi-text.tistory.com/2 파라미터 정리 
# size : 벡터 차원수 ,
# window : 훈련시 앞 뒤로 고려하는 단어의 개수
# min_count :최소빈도수 , 해당 빈도수보다 작게 등장한 단어 제외
# worker : 병렬처리할 스레드 수 
#sg : 분석방법론 , 1 = skip-gram, 0=cbow
#iter 학습횟수

from gensim.models import Word2Vec

#skip gram
model2=Word2Vec(tokenized_data, vector_size=100, window=3, min_count=5,workers=4, sg=1,epochs=100)

In [14]:
print(model2.wv.most_similar("환경"))
print(model2.wv.most_similar("지배구조"))
print(model2.wv.most_similar("사회"))

#모델 저장

model2.wv.save_word2vec_format('news_skipgram_w2v') # 모델 저장
loaded_model2 = KeyedVectors.load_word2vec_format("news_skipgram_w2v") # 모델 로드

[('변화', 0.5579166412353516), ('민첩성', 0.5516247153282166), ('탄력성', 0.5371886491775513), ('백오피스', 0.5367226600646973), ('여건', 0.5285210013389587), ('스마트워크', 0.5277842283248901), ('인프라', 0.5199689269065857), ('악천후', 0.5175926685333252), ('오염시키다', 0.5136458873748779), ('안전하다', 0.5121936202049255)]
[('경영', 0.6901301741600037), ('사회', 0.5772139430046082), ('지배', 0.5631550550460815), ('책임', 0.545724093914032), ('지주회사', 0.5374337434768677), ('구조', 0.5149639248847961), ('지주사', 0.5120095014572144), ('차입', 0.5014294981956482), ('건전성', 0.49147123098373413), ('폐쇄적', 0.4893702268600464)]
[('공헌', 0.6660206317901611), ('활동', 0.6133454442024231), ('대타협', 0.6041689515113831), ('나눔', 0.6019120812416077), ('지배구조', 0.5772138237953186), ('실천', 0.5711150169372559), ('문미숙', 0.545948326587677), ('국제사회', 0.5429704785346985), ('지역', 0.539405107498169), ('책임', 0.5369611382484436)]


In [ ]:
words1 = list(model.wv.vocab)
print(words1)
words2 = list(model2.wv.vocab)
print(words2)

- news, jobplanet, dart , patent text 데이터로 단어집 만들기

In [1]:
#데이터 불러오기
import pandas as pd
news=pd.read_csv('./news_tb.csv')
jobplanet=pd.read_csv('./jobplanet_tb.csv')
dart=pd.read_csv('./dart_tb.csv')
patent=pd.read_csv('./patent_tb.csv')
all=pd.concat([news['Content'],jobplanet['Review'],dart['EmphsMatter'],patent['Content']])

In [2]:

all_df=pd.DataFrame(all)
all_df.columns = ['contents']
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93414 entries, 0 to 10295
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   contents  92247 non-null  object
dtypes: object(1)
memory usage: 1.4+ MB


In [3]:
print(all_df.isnull().values.any()) #결측값 있음, 숫자, 영어 제거된 상태
print(all_df.isnull().sum()) 

True
contents    1167
dtype: int64


In [4]:
all_df=all_df.dropna(axis=0)
print(all_df.isnull().sum()) 

contents    0
dtype: int64


In [5]:
#로컬에 저장한 mecab 파일 경로 지정해주기
# 형태소분석기 mecab 활용 토큰화 작업
 
from konlpy.tag import Okt, Mecab
from tqdm import tqdm
from gensim.models.word2vec import Word2Vec

#okt = Okt()
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

tokenized_data = []
for sentence in tqdm(all_df['contents']):
    if len(sentence) > 1:
        tokenized_sentence = mecab.morphs(sentence) # 토큰화
        tokenized_data.append(tokenized_sentence)

100%|██████████| 92247/92247 [01:34<00:00, 975.57it/s] 


In [6]:
# 토큰화된 뉴스 데이터 학습시키기
#https://hoonzi-text.tistory.com/2 파라미터 정리 
# size : 벡터 차원수 ,
# window : 훈련시 앞 뒤로 고려하는 단어의 개수
# min_count :최소빈도수 , 해당 빈도수보다 작게 등장한 단어 제외
# worker : 병렬처리할 스레드 수 
#sg : 분석방법론 , 1 = skip-gram, 0=cbow
#iter or epochs : 학습횟수

from gensim.models import Word2Vec

#cbow 
model=Word2Vec(tokenized_data, vector_size=100, window=3, min_count=5,workers=4, sg=0,epochs=100) #window 3
model_2=Word2Vec(tokenized_data, vector_size=100, window=5, min_count=5,workers=4, sg=0,epochs=100) #window 5

e_친환경=model.wv.most_similar("친환경",topn=50)
e_환경=model.wv.most_similar("환경",topn=50)
s_사회=model.wv.most_similar("사회",topn=50)
s_근로=model.wv.most_similar("근로",topn=50)
g_경영=model.wv.most_similar("경영",topn=50)
g_지배=model.wv.most_similar("지배",topn=50)


e_친환경2=model_2.wv.most_similar("친환경",topn=50)
e_환경2=model_2.wv.most_similar("환경",topn=50)
s_사회2=model_2.wv.most_similar("사회",topn=50)
s_근로2=model_2.wv.most_similar("근로",topn=50)
g_경영2=model_2.wv.most_similar("경영",topn=50)
g_지배2=model_2.wv.most_similar("지배",topn=50)


In [7]:

def make_dict(df):
    temp = []
    for word, percent in df:
        temp.append(word)
    df = temp
    return df

#window 3

e_친환경df=make_dict(e_친환경)
e_환경df=make_dict(e_환경)
s_근로df=make_dict(s_근로)
s_사회df=make_dict(s_사회)
g_지배df=make_dict(g_지배)
g_경영df=make_dict(g_경영)


esg_df=pd.DataFrame(zip(e_친환경df, e_환경df,s_근로df,s_사회df,g_지배df,g_경영df))
esg_df.columns=['e_친환경','e_환경','s_근로','s_사회','g_지배','g_경영']
esg_df.to_csv('./all_esg_dict_mecab_win3_ver2.csv')


#window 5

e_친환경df2=make_dict(e_친환경2)
e_환경df2=make_dict(e_환경2)
s_근로df2=make_dict(s_근로2)
s_사회df2=make_dict(s_사회2)
g_지배df2=make_dict(g_지배2)
g_경영df2=make_dict(g_경영2)


esg_df2=pd.DataFrame(zip(e_친환경df2, e_환경df2,s_근로df2,s_사회df2,g_지배df2,g_경영df2))
esg_df2.columns=['e_친환경','e_환경','s_근로','s_사회','g_지배','g_경영']
esg_df2.to_csv('./all_esg_dict_mecab_win5_ver3.csv')

In [8]:
# skip gram 

model_3=Word2Vec(tokenized_data, vector_size=100, window=5, min_count=5,workers=4, sg=1,epochs=100)

e_친환경3=model_3.wv.most_similar("친환경",topn=50)
e_환경3=model_3.wv.most_similar("환경",topn=50)
s_사회3=model_3.wv.most_similar("사회",topn=50)
s_근로3=model_3.wv.most_similar("근로",topn=50)
g_경영3=model_3.wv.most_similar("경영",topn=50)
g_지배3=model_3.wv.most_similar("지배",topn=50)

def make_dict(df):
    temp = []
    for word, percent in df:
        temp.append(word)
    df = temp
    return df


e_친환경df3=make_dict(e_친환경3)
e_환경df3=make_dict(e_환경3)
s_근로df3=make_dict(s_근로3)
s_사회df3=make_dict(s_사회3)
g_지배df3=make_dict(g_지배3)
g_경영df3=make_dict(g_경영3)


esg_df3=pd.DataFrame(zip(e_친환경df3, e_환경df3,s_근로df3,s_사회df3,g_지배df3,g_경영df3))
esg_df3.columns=['e_친환경','e_환경','s_근로','s_사회','g_지배','g_경영']
esg_df3.to_csv('./all_esg_dict_mecab_skip_ver4.csv')

### FastText 로 e,s,g 단어집 만들기
- cbow, skip gram 둘다 진행
- e : 환경, 친환경 
- s : 사회, 근로
- g : 경영, 지배 

In [9]:
from gensim.models.fasttext import FastText

model_4 = FastText(tokenized_data, window=5, min_count=5, sg=0) #cbow
#model_4 = FastText(tokenized_data, window=5, min_count=5, sg=1)# skip gram

e_친환경4=model_4.wv.most_similar("친환경",topn=50)
e_환경4=model_4.wv.most_similar("환경",topn=50)
s_사회4=model_4.wv.most_similar("사회",topn=50)
s_근로4=model_4.wv.most_similar("근로",topn=50)
g_경영4=model_4.wv.most_similar("경영",topn=50)
g_지배4=model_4.wv.most_similar("지배",topn=50)

def make_dict(df):
    temp = []
    for word, percent in df:
        temp.append(word)
    df = temp
    return df


e_친환경df4=make_dict(e_친환경4)
e_환경df4=make_dict(e_환경4)
s_근로df4=make_dict(s_근로4)
s_사회df4=make_dict(s_사회4)
g_지배df4=make_dict(g_지배4)
g_경영df4=make_dict(g_경영4)


esg_df4=pd.DataFrame(zip(e_친환경df4, e_환경df4,s_근로df4,s_사회df4,g_지배df4,g_경영df4))
esg_df4.columns=['e_친환경','e_환경','s_근로','s_사회','g_지배','g_경영']
esg_df4.to_csv('./all_esg_dict_mecab_fast_cbow.csv')